<a href="https://colab.research.google.com/github/appletreeleaf/Algorithms/blob/main/%5BHW27_Problem%5D_LSTM%26GRU_%EC%9D%B4%EC%9E%AC%EC%98%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [HW27] LSTM&GRU



*   본 실습에서는 Pytorch 를 활용한 LSTM 사용법을 학습합니다.

> Reference: https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html


### Part 1. LSTM with toy example

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

`torch.nn` 을 활용하여 LSTM cell 을 생성하는 방법은 다음과 같습니다.

* `input_size` : The number of expected features in the input x

* `hidden_size` : The number of features in the hidden state h
```
lstm = nn.LSTM(input_size, hidden_size)
```



In [ ]:
# input_size: 3, hidden_size: 3 으로 설정하여 LSTM cell 을 생성합니다.
lstm = nn.LSTM(3, 3)

LSTM cell 을 생성한 후에는, 입력으로 들어갈 input x, hidden state h, cell state c 를 생성해야 합니다.

위에서 정한 input_size 와 hidden_size 를 고려하여 inputs 와 hidden (h 와 c) 을 생성해 봅시다.

In [ ]:
# sequence length 가 5 인 input을 생성합니다.
# 이때, input_size 를 3 으로 설정했으므로, 3 차원 벡터 5개를 생성해야 합니다.
inputs = [torch.randn(1, 3) for _ in range(5)]

# lstm 은 input x 와 hidden state h 를 입력으로 받기 때문에, hidden state 도 생성해 줍니다.
# 이때, hidden_size 를 3 으로 설정했으므로, 3 차원 벡터를 생성합니다.
# lstm 의 입력으로 들어가는 h 는 RNN 에서의 hidden state 와, lstm 에서 등장한 개념인 cell state 로 구성되어 있기 때문에
# hidden 은 3 차원 벡터 2개로 구성되어야 합니다.
hidden = (torch.randn(1, 1, 3),  # 1개는 h, 한개는 c
          torch.randn(1, 1, 3))

**방법 1**: Sequence length 가 5 인 input 에 대하여 한 번에 하나의 element 를 lstm cell 에 통과시킵니다.

In [ ]:
for idx, i in enumerate(inputs):
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)
    print(f"Time step {idx}, out is : ", out)
    print(f"Time step {idx}, hidden is : ", hidden)
    print()


Time step 0, out is :  tensor([[[-0.2682,  0.0304, -0.1526]]], grad_fn=<StackBackward0>)
Time step 0, hidden is :  (tensor([[[-0.2682,  0.0304, -0.1526]]], grad_fn=<StackBackward0>), tensor([[[-1.0766,  0.0972, -0.5498]]], grad_fn=<StackBackward0>))

Time step 1, out is :  tensor([[[-0.5370,  0.0346, -0.1958]]], grad_fn=<StackBackward0>)
Time step 1, hidden is :  (tensor([[[-0.5370,  0.0346, -0.1958]]], grad_fn=<StackBackward0>), tensor([[[-1.1552,  0.1214, -0.2974]]], grad_fn=<StackBackward0>))

Time step 2, out is :  tensor([[[-0.3947,  0.0391, -0.1217]]], grad_fn=<StackBackward0>)
Time step 2, hidden is :  (tensor([[[-0.3947,  0.0391, -0.1217]]], grad_fn=<StackBackward0>), tensor([[[-1.0727,  0.1104, -0.2179]]], grad_fn=<StackBackward0>))

Time step 3, out is :  tensor([[[-0.1854,  0.0740, -0.0979]]], grad_fn=<StackBackward0>)
Time step 3, hidden is :  (tensor([[[-0.1854,  0.0740, -0.0979]]], grad_fn=<StackBackward0>), tensor([[[-1.0530,  0.1836, -0.1731]]], grad_fn=<StackBackward0>

**방법 2**: 전체 시퀀스를 한번에 통과시키는 방법도 있습니다.

LSTM 이 반환하는 출력의 첫 번째 값은 전체 시퀀스에 대한 통과한 hidden state 이고, 두 번째 값은 마지막 step 의 hidden state 입니다.

`out` 과 `hidden` 의 size 를 비교해보세요.

In [ ]:
inputs = torch.cat(inputs).view(len(inputs), 1, -1) # 방법 2 를 적용하기 위해 input 을 list 가 아닌 하나의 tensor 로 concat 해줍니다.
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # 방법 2 를 적용하기 위해 hidden 을 다시 초기화합니다.
out, (h_n, c_n) = lstm(inputs, hidden)
print(out)
print("h_n is :",h_n) # final time step의 hidden state
print("c_n is :",c_n) # "" cell

tensor([[[-0.0187,  0.1713, -0.2944]],

        [[-0.3521,  0.1026, -0.2971]],

        [[-0.3191,  0.0781, -0.1957]],

        [[-0.1634,  0.0941, -0.1637]],

        [[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward0>)
h_n is : tensor([[[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward0>)
c_n is : tensor([[[-0.9825,  0.4715, -0.0633]]], grad_fn=<StackBackward0>)


### Part 2. LSTM for Part-of-Speech Tagging

LSTM 을 이용해 Part-of-Speech (PoS) Tagging 을 하기 위해 학습 데이터를 준비합니다.

* `training_data` 에는 단어 시퀀스와 각 단어의 품사 태그를 준비해야 합니다.

* `word_to_ix`: 모델의 입력으로 사용하기 위해 각 단어를 id 로 mapping 합니다.

* `tag_to_ix`: 품사 태그 또한 id 로 mapping 합니다.

In [ ]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long) # word to index 변환한 list


training_data = [
    # Tags are: DET - determiner; NN - noun; V - verb
    # For example, the word "The" is a determiner
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]

word_to_ix = {}
# For each words-list (sentence) and tags-list in each tuple of training_data
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:  # word has not been assigned an index yet
            word_to_ix[word] = len(word_to_ix)  # Assign each word with a unique index
print("word_to_ix is : ",word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}  # Assign each tag with a unique index

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6 # input_dim
HIDDEN_DIM = 6    # hidden_vec_dim

word_to_ix is :  {'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


Embedding layer, output layer, lstm cell 을 포함한 LSTMTagger 모듈을 정의합니다.

`embeds`: input id 를 embedding layer 로 encode 하여 input 에 해당하는 embedding 생성합니다.

`lstm_out`: embedding 을 lstm 에 통과하여 전체 시퀀스에 대한 hidden state 를 저장합니다.

`tag_space`: lstm 의 output 인 hidden 을 이용해 존재하는 tag (DET, NN, V) 공간으로 linear transform 합니다.

`tag_scores`: 이후 softmax 를 적용하여 각 tag 가 될 score 를 측정합니다.

In [ ]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim) #  vocab size에 대응되는 one-hot vector를 지정된 embedding dim으로 축소한다다

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim) # hidden_size=(1,1,hidden_dim) / 6차원 벡터

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence) # LSTM의 input
        print("embeding_vec : ",embeds)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1)) # h, (h_n, c_n)
        print("lstm_out : ", lstm_out) # h_1~n
        print("h_n, c_n : ", _)        # h_n, c_n
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1)) # hidden state를 tagset_size로 linear transformation
        print("hidden to tag : ",tag_space)
        tag_scores = F.log_softmax(tag_space, dim=1) # softmax로 각 원소의 확률값을 구함
        return tag_scores

정의한 LSTMTagger 클래스를 이용해 model 을 build 하고, 학습에 필요한 loss 함수와 optimizer 를 선언합니다.

In [ ]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

모델을 학습하기 전 tag score 를 확인해봅시다.

In [ ]:
# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
# Current prediction 2 2 2 2 1, not correct...!
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)


embeding_vec :  tensor([[ 0.7575, -0.4068, -0.1277,  0.2804,  0.0375, -0.6378],
        [-0.8148, -0.6895,  0.7705, -1.0739, -0.2015, -0.5603],
        [ 0.6817, -0.5170,  1.7902,  0.5877,  0.2505, -0.7930],
        [ 0.5231,  1.2236, -0.9458, -1.3529,  3.3837, -2.4044],
        [-0.3891, -0.0796,  0.7605, -1.0025,  0.9462,  0.3512]])
lstm_out :  tensor([[[ 0.0285,  0.0525, -0.1087, -0.1412,  0.1198,  0.0263]],

        [[ 0.0270,  0.1370, -0.1445,  0.2196,  0.0025, -0.0633]],

        [[ 0.0197,  0.1615, -0.1146, -0.0657,  0.0950, -0.1137]],

        [[-0.0440,  0.1359, -0.3066, -0.0893,  0.3248, -0.2291]],

        [[-0.2109,  0.1769, -0.1864,  0.1180,  0.0328, -0.4014]]])
h_n, c_n :  (tensor([[[-0.2109,  0.1769, -0.1864,  0.1180,  0.0328, -0.4014]]]), tensor([[[-0.3178,  0.5583, -0.4210,  0.2104,  0.0472, -0.7124]]]))
hidden to tag :  tensor([[ 0.0478, -0.0156,  0.2175],
        [ 0.0137, -0.0997,  0.1369],
        [ 0.0469, -0.0338,  0.2029],
        [ 0.1122,  0.0353,  0.2397],
  

이제, training data 를 이용해 모델을 학습합니다. 즉, input 을 LSTMTagger 에 통과시켜 각 단어의 PoS tag 를 예측하고, 정답 tag 와 비교하여 loss 를 계산한 후 loss 를 backpropagate 하여 모델 파라미터를 업데이트 합니다.

In [ ]:

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad() # gradient_initialization

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix) #embeding input (X)
        targets = prepare_sequence(tags, tag_to_ix) #embeding target (y)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
embeding_vec :  tensor([[ 1.4336,  1.7054,  0.1073, -2.6362,  0.2071,  0.2167],
        [-0.0888,  0.1554,  1.3069,  1.2114,  1.1115,  0.9785],
        [-0.6286,  1.5422,  0.1610,  2.0367, -0.3404, -0.8157],
        [-1.0857,  1.2293,  1.4159, -0.7446,  0.2079, -0.5774]],
       grad_fn=<EmbeddingBackward0>)
lstm_out :  tensor([[[-0.1799,  0.2589, -0.2256,  0.6473,  0.0823, -0.1867]],

        [[ 0.1923,  0.1891, -0.2344, -0.5611, -0.6498, -0.3887]],

        [[ 0.0573, -0.0067,  0.3869, -0.7192,  0.6166, -0.1598]],

        [[-0.3370,  0.3345,  0.3662,  0.6801, -0.1550, -0.4551]]],
       grad_fn=<StackBackward0>)
h_n, c_n :  (tensor([[[-0.3370,  0.3345,  0.3662,  0.6801, -0.1550, -0.4551]]],
       grad_fn=<StackBackward0>), tensor([[[-0.4687,  0.5358,  0.4823,  0.8539, -0.1825, -0.7953]]],
       grad_fn=<StackBackward0>))
hidden to tag :  tensor([[-1.3042,  2.2874, -1.2020],
        [-0.4734, -0.6872,  2.0110],
        [ 2.4268, -1.4952, -0.6555]

학습 데이터를 이용해 모델이 잘 학습되었는지 확인해봅시다. 학습하기 전 tag score 와 비교해봅니다.

In [ ]:
# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j // i번째 단어가 j번째 태그일 확률 matrix
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print("tag_scores : "tag_scores)

embeding_vec :  tensor([[ 7.3412e-01,  8.0676e-04, -3.3758e-01,  6.2446e-01,  2.0486e-01,
         -1.0973e+00],
        [-9.2287e-01, -7.1421e-01,  9.6096e-01, -1.2692e+00, -1.8903e-01,
         -6.0069e-01],
        [ 6.4489e-01, -6.8376e-01,  2.1375e+00,  6.7106e-01,  3.9116e-01,
         -7.5065e-01],
        [ 4.5504e-01,  1.3352e+00, -9.5383e-01, -1.2280e+00,  3.4942e+00,
         -2.5017e+00],
        [-4.8974e-01, -5.7369e-02,  9.1785e-01, -1.1828e+00,  9.0761e-01,
          3.0940e-01]])
lstm_out :  tensor([[[-0.0040, -0.0062,  0.3183, -0.5976,  0.5878,  0.0014]],

        [[-0.2208,  0.3527, -0.1587,  0.6699, -0.1560, -0.2218]],

        [[ 0.4055,  0.3747, -0.3509, -0.6701, -0.6923, -0.3002]],

        [[-0.2733,  0.1153,  0.5814, -0.7665,  0.6997, -0.3601]],

        [[-0.4518,  0.5708, -0.1789,  0.6689, -0.2842, -0.6710]]])
h_n, c_n :  (tensor([[[-0.4518,  0.5708, -0.1789,  0.6689, -0.2842, -0.6710]]]), tensor([[[-0.7351,  1.3520, -0.2775,  0.8572, -0.3727, -1.2617]]]))
hi

### Questions.
1. LSTM 이 아닌 GRU 를 사용하려면 본 실습 코드의 부분을 바꿔야 할까요? Part 2 부분을 GRU 로 대체하여 PoS Tagger 를 학습하고, LSTM 과의 차이를 비교해 봅시다.
2. 본 실습에서는 학습 데이터만을 이용해 모델을 학습하고, 평가합니다. 이때 발생할 수 있는 문제점은 어떤 것이 있을까요? 그 문제점을 해결하려면 어떻게 해야 할까요? $->$ test 과정이 빠져있으므로 overfitting 문제를 확인하기 어렵다. 이로인해 모델의 성능을 측정하기 어렵다.